In [2]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

We want to eliminate "accidental" referrals, so filter the hop teaming data so that the transaction_count is at least 50 and the average_day_wait is less than 50.

In [ ]:
##db = sqlite3.connect('data/doc_grahp_hop.sqlite')

##chunks = pd.read_csv('data/DocGraph_Hop_Teaming_2018.csv', chunksize = 10000)
##hop = []
##for chunk in chunks: 
##    chunk = chunk[(chunk['transaction_count'] >= 50) & (chunk['average_day_wait'] < 50)]
##    hop.append(chunk)
##    chunk.to_sql('hop', db, if_exists = 'append', index = False) 

In [ ]:
##hopp = pd.concat(hop, ignore_index = True)
##hopp

In [3]:
nppes_1 = pd.read_csv('data/NPPES_Data_Dissemination_February_2023/npidata_pfile_20050523-20230212.csv', nrows=100)
nppes_1

,NPI,Entity Type Code,Replacement NPI,Employer Identification Number (EIN),Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,...,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15,Certification Date
0,1740284231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1346245800,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1487650776,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1033113022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1043216138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1538164819,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,1295732865,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,1861497844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,1043217565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
nppes_1.loc[:, nppes_1.columns.str.startswith("Healthcare Provider Taxonomy")]

,Healthcare Provider Taxonomy Code_1,Healthcare Provider Taxonomy Code_2,Healthcare Provider Taxonomy Code_3,Healthcare Provider Taxonomy Code_4,Healthcare Provider Taxonomy Code_5,Healthcare Provider Taxonomy Code_6,Healthcare Provider Taxonomy Code_7,Healthcare Provider Taxonomy Code_8,Healthcare Provider Taxonomy Code_9,Healthcare Provider Taxonomy Code_10,...,Healthcare Provider Taxonomy Group_6,Healthcare Provider Taxonomy Group_7,Healthcare Provider Taxonomy Group_8,Healthcare Provider Taxonomy Group_9,Healthcare Provider Taxonomy Group_10,Healthcare Provider Taxonomy Group_11,Healthcare Provider Taxonomy Group_12,Healthcare Provider Taxonomy Group_13,Healthcare Provider Taxonomy Group_14,Healthcare Provider Taxonomy Group_15
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
nppes_1.loc[:, nppes_1.columns.str.startswith("Healthcare Provider Primary Taxonomy Switch{i}" == "Y")]

KeyError: "None of [Float64Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              ...\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n             dtype='float64', length=330)] are in the [columns]"

In [119]:
health = []
for i in range(1,16):
    code = (f'Healthcare Provider Taxonomy{i}')
    switch = (f'Healthcare Provider Primary Taxonomy Switch{i}')
    health.append(code)
    health.append(switch)
    
nppes_cols = []
nppes_col =({
   "npi",
    "provider_organization_name",
    "provider_last_name",
    "provider_first_name",
    "provider_middle_name",
    "provider_name_prefix_text",
    "provider_name_suffix_text",
    "provider_credential_text"
})
nppes_cols.append(nppes_col)


TypeError: unsupported operand type(s) for +: 'set' and 'list'

In [117]:
nppes_cols = []
nppes_col =({
   "npi",
    "provider_organization_name",
    "provider_last_name",
    "provider_first_name",
    "provider_middle_name",
    "provider_name_prefix_text",
    "provider_name_suffix_text",
    "provider_credential_text" 
})
nppes_cols.append(nppes_col)


In [6]:
taxonomy = pd.read_csv('data/nucc_taxonomy_230.csv', nrows=100)
taxonomy

,Code,Grouping,Classification,Specialization,Definition,Notes,Display Name,Section
0,193200000X,Group,Multi-Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Multi-Specialty Group,Individual
1,193400000X,Group,Single Specialty,NaN,A business group of one or more individual pra...,[7/1/2003: new],Single Specialty Group,Individual
2,207K00000X,Allopathic & Osteopathic Physicians,Allergy & Immunology,NaN,An allergist-immunologist is trained in evalua...,"Source: American Board of Medical Specialties,...",Allergy & Immunology Physician,Individual
3,207KA0200X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Allergy,"A physician who specializes in the diagnosis, ...",Source: National Uniform Claim Committee,Allergy Physician,Individual
4,207KI0005X,Allopathic & Osteopathic Physicians,Allergy & Immunology,Clinical & Laboratory Immunology,An allergy and immunology physician who specia...,"Source: National Uniform Claim Committee, 2022...",Clinical & Laboratory Immunology (Allergy & Im...,Individual
...,...,...,...,...,...,...,...,...
95,207WX0120X,Allopathic & Osteopathic Physicians,Ophthalmology,Cornea and External Diseases Specialist,An ophthalmologist who specializes in diseases...,"Source: American Academy of Ophthalmology, www...",Cornea and External Diseases Specialist Physician,Individual
96,207WX0009X,Allopathic & Osteopathic Physicians,Ophthalmology,Glaucoma Specialist,An ophthalmologist who specializes in the trea...,"Source: American Academy of Ophthalmology, www...",Glaucoma Specialist (Ophthalmology) Physician,Individual
97,207WX0109X,Allopathic & Osteopathic Physicians,Ophthalmology,Neuro-ophthalmology,A neuro-ophthalmologist is a subspecialist of ...,"Source: American Academy of Ophthalmology, www...",Neuro-ophthalmology Physician,Individual
98,207WX0200X,Allopathic & Osteopathic Physicians,Ophthalmology,Ophthalmic Plastic and Reconstructive Surgery,A physician who specializes in oculofacial pla...,"Source: American Academy of Ophthalmology, 20...",Ophthalmic Plastic and Reconstructive Surgery ...,Individual


In [ ]:
##db = sqlite3.connect('data/NPPES_Data_Dissemination.sqlite')

##chunks = pd.read_csv('data/NPPES_Data_Dissemination_February_2023.csv', chunksize = 10000)
##nppes = []
##for chunk in chunks: 

##    hop.append(chunk)